# Outliers 

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.feature_selection import RFE
from sklearn.model_selection import cross_val_score
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error,mean_absolute_error,r2_score

import os
import time
import warnings

warnings.filterwarnings("ignore")

In [ ]:
df = pd.read_excel(r"C:\Users\HIT\Desktop\5000_papers.xlsx")
testdata = pd.read_excel(r"C:\Users\HIT\Desktop\143new_data_papers(1).xlsx")

In [ ]:
data_first = df.loc[0:,:]
type_ = data_first[['Type']]
type_mapping = {
           'low temperture': 0,
           'room temperature': 1,
           'medium temperature': 2,
           'high temperature': 3 }
type_['Type'] = type_['Type'].map(type_mapping)

In [ ]:
df1 = data_first[['Material Group','Lattice form','Method']]
data_first = data_first.drop('Type',axis = 1,inplace = False)
data_first = data_first.drop('S（uV/K）',axis = 1,inplace = False)
data_first = data_first.drop('ρ（μΩm）',axis=1,inplace=False)
data_first = data_first.drop('κ（W/mK）',axis=1,inplace=False)
data_first = data_first.drop('Formula',axis = 1,inplace = False)

In [ ]:
zt = data_first.pop('ZT')
data = pd.concat([zt,data_first],axis = 1)

In [ ]:
def find_outliers(model, X, y, sigma=3):
    
    #使用模型预测y值
    try:
        y_pred = pd.Series(model.predict(X), index=y.index)
    #如果无法预测，则先使模型拟合训练集
    except:
        model.fit(X,y,categorical_feature=category_feature)
        y_pred = pd.Series(model.predict(X), index=y.index)
    
    #计算模型预测y值与真实y值之间的残差
    resid = y - y_pred
    mean_resid = resid.mean()
    std_resid = resid.std()

    #计算异常值定义的参数z参数,数据的|z|大于σ将会被视为异常
    z = (resid - mean_resid) / std_resid
    outliers = z[abs(z) > sigma].index
    
    #打印结果并绘制图像
    print('R2 = ',model.score(X,y))
    print('MSE = ',mean_squared_error(y, y_pred))
    print('------------------------------------------')
    
    print('mean of residuals',mean_resid)
    print('std of residuals',std_resid)
    print('------------------------------------------')
    
    print(f'find {len(outliers)}','outliers：')
    print(outliers.tolist())
    
    
    plt.figure(figsize=(15,5))
    ax_131 = plt.subplot(1,3,1)
    plt.plot(y,y_pred,'.')
    plt.plot(y.loc[outliers],y_pred.loc[outliers],'ro')
    plt.legend(['Accepted','Outliers'])
    plt.xlabel('y')
    plt.ylabel('y_pred');
    
    ax_132 = plt.subplot(1,3,2)
    plt.plot(y, y-y_pred, '.')
    plt.plot(y.loc[outliers],y.loc[outliers] - y_pred.loc[outliers],'ro')
    plt.legend(['Accepted','Outliers'])
    plt.xlabel('y')
    plt.ylabel('y - y_pred');
    
    ax_133 = plt.subplot(1,3,3)
    z.plot.hist(bins=50, ax=ax_133)
    z.loc[outliers].plot.hist(color='r', bins=50, ax=ax_133)
    plt.legend(['Accepted','Outliers'])
    plt.xlabel('z')
    
    plt.savefig('logs/outliers.png',dpi=200)
    
    return outliers

In [ ]:
category_feature=['Material Group','Lattice form','Method']

# Feature engineering and selection

In [ ]:
import lightgbm as lgb

In [ ]:
from sklearn import preprocessing
lbl = preprocessing.LabelEncoder()
for i in category_feature:
    data[i] = lbl.fit_transform(data[i].astype(str))#将提示的包含错误数据类型这一列进行转换

In [ ]:
from sklearn.feature_selection import RFE
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error,mean_absolute_error,r2_score
from lightgbm import LGBMRegressor
X_train = data.iloc[:,1:]
y_train = data.iloc[:,0]
outliers = find_outliers(LGBMRegressor(n_estimators=1000), X_train, y_train)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import SGDRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost.sklearn import XGBRegressor
from lightgbm import LGBMRegressor


from sklearn.model_selection import GridSearchCV
import joblib
from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score
from sklearn.model_selection import KFold,train_test_split

from sklearn.model_selection import ShuffleSplit, learning_curve

import os
import time
import warnings

warnings.filterwarnings('ignore')

In [ ]:
df = pd.concat([df,testdata],axis = 0)
df.index = range(df.shape[0])
df_1 = df.loc[0:,:]
type_ = df[['Type']]
type_mapping = {
           'low temperture': 0,
           'room temperature': 1,
           'medium temperature': 2,
           'high temperature': 3 }
type_['Type'] = type_['Type'].map(type_mapping)
from sklearn import preprocessing
lbl = preprocessing.LabelEncoder()
for i in category_feature:
    df_1[i] = lbl.fit_transform(df_1[i].astype(str))

In [ ]:
df1 = df[['Material Group','Lattice form','Method']]
df_1 = df_1.drop('Type',axis = 1,inplace = False)
df_1 = df_1.drop('S（uV/K）',axis = 1,inplace = False)
formula = df_1.pop('Formula')
df_zt = df_1.pop('ZT')
df_zt

In [ ]:
df1 = df_1[['Material Group','Lattice form','Method']]
df_1 = df_1.drop('Material Group',axis=1,inplace=False)
df_1 = df_1.drop('Method',axis=1,inplace=False)
df_1 = df_1.drop('Lattice form',axis=1,inplace=False)

In [ ]:
for i in range(df_1.shape[0]):
    df_1.iloc[i,1:] = df_1.iloc[i,1:]*df_1['T（K)'][i]
df_1 = pd.concat([df_zt,df_1],axis = 1)

from sklearn.preprocessing import MinMaxScaler

features_columns = [col for col in df_1.columns if col not in ['ZT']]
min_max_scaler = MinMaxScaler()
min_max_scaler = min_max_scaler.fit(df_1[features_columns])
df_1_scaler = min_max_scaler.transform(df_1[features_columns])

df_1_scaler = pd.DataFrame(df_1_scaler)
df_1_scaler.columns = features_columns


# df_1_scaler['ZT'] = df_1['ZT']

df_1_scaler = pd.concat([df_zt,df_1_scaler],axis = 1)

In [ ]:
col = [
 'ZT',
 'T（K)',
 'mean Number',
 'minimum Number','maximum Number','range Number','mode Number','avg_dev Number',
 
 'mean MendeleevNumber','minimum MendeleevNumber','maximum MendeleevNumber',
 'range MendeleevNumber','mode MendeleevNumber','avg_dev MendeleevNumber',
 
 'mean AtomicWeight','minimum AtomicWeight','maximum AtomicWeight',
 'range AtomicWeight','mode AtomicWeight','avg_dev AtomicWeight',
    
 'mean MeltingT','minimum MeltingT','maximum MeltingT','range MeltingT','mode MeltingT',
 'avg_dev MeltingT',
    
 'mean Column','minimum Column','maximum Column','range Column','mode Column',
 'avg_dev Column',
    
 'mean Row','minimum Row','maximum Row','range Row','mode Row',
 'avg_dev Row',
    
 'mean CovalentRadius','minimum CovalentRadius','maximum CovalentRadius','range CovalentRadius',
 'mode CovalentRadius','avg_dev CovalentRadius',
 
 'mean Electronegativity','minimum Electronegativity','maximum Electronegativity',
 'range Electronegativity','mode Electronegativity','avg_dev Electronegativity',
    
 'mean NsValence','minimum NsValence','maximum NsValence','range NsValence','mode NsValence',
 'avg_dev NsValence',
    
 'mean NpValence','minimum NpValence','maximum NpValence','range NpValence','mode NpValence',
 'avg_dev NpValence',
    
 'mean NdValence','minimum NdValence','maximum NdValence','range NdValence','mode NdValence',
 'avg_dev NdValence',
    
 'mean NfValence','minimum NfValence','maximum NfValence','range NfValence','mode NfValence',
 'avg_dev NfValence',
    
 'mean NValence','minimum NValence','maximum NValence','range NValence','mode NValence',
 'avg_dev NValence',
    
 'mean NsUnfilled','minimum NsUnfilled','maximum NsUnfilled','range NsUnfilled','mode NsUnfilled',
 'avg_dev NsUnfilled',
    
 'mean NpUnfilled','minimum NpUnfilled','maximum NpUnfilled','range NpUnfilled','mode NpUnfilled',
 'avg_dev NpUnfilled',
    
 'mean NdUnfilled','minimum NdUnfilled','maximum NdUnfilled','range NdUnfilled','mode NdUnfilled',   
 'avg_dev NdUnfilled',
    
 'mean NfUnfilled','minimum NfUnfilled','maximum NfUnfilled','range NfUnfilled','mode NfUnfilled',
 'avg_dev NfUnfilled',
    
 'mean NUnfilled','minimum NUnfilled','maximum NUnfilled','range NUnfilled','mode NUnfilled',
 'avg_dev NUnfilled',
    
 'mean GSvolume_pa','minimum GSvolume_pa','maximum GSvolume_pa','range GSvolume_pa','mode GSvolume_pa',
 'avg_dev GSvolume_pa',
    
 'mean GSbandgap','minimum GSbandgap','maximum GSbandgap','range GSbandgap','mode GSbandgap',
 'avg_dev GSbandgap',
    
 'mean SpaceGroupNumber','minimum SpaceGroupNumber','maximum SpaceGroupNumber','range SpaceGroupNumber',
 'mode SpaceGroupNumber','avg_dev SpaceGroupNumber',
    
 'mean heat_fusion','minimum heat_fusion','maximum heat_fusion','range heat_fusion',
 'std_dev heat_fusion',
    
 'mean boiling_point','minimum boiling_point','maximum boiling_point','range boiling_point',
 'std_dev boiling_point',
    
 'mean first_ioniz','minimum first_ioniz','maximum first_ioniz','range first_ioniz',
 'std_dev first_ioniz',
    
 'mean block','minimum block','maximum block','range block',
 'std_dev block',
    
 'mean thermal_conductivity','minimum thermal_conductivity','maximum thermal_conductivity',
 'range thermal_conductivity','std_dev thermal_conductivity',
    
 'mean electron_affinity','mean lattice_constant', 'mean specific_heat','mean dipole_polarizability',
 'mean atomic_volume','mean vdw_radius','mean density',
 
 'std_dev atomic_volume','std_dev vdw_radius','std_dev density','std_dev lattice_constant',
 'std_dev specific_heat','std_dev electron_affinity','std_dev dipole_polarizability',
    
 'entropy atomic_volume','entropy vdw_radius','entropy density','entropy lattice_constant',
 'entropy specific_heat','entropy dipole_polarizability',
    
 'Gmean atomic_volume','Gmean vdw_radius','Gmean density','Gmean lattice_constant','Gmean specific_heat',
 'Gmean dipole_polarizability','nelement'
 
 ]

In [ ]:
k = ['PymatgenData','DemlData','MagpieData']
for i in range(len(df_1_scaler.columns)):
    s = df_1_scaler.columns[i]
    s = s.split(' ')
    if s[0] in k:
        s = s[1:]
        s = ' '.join(s)
        df_1_scaler.rename(columns={df_1_scaler.columns[i]:s},inplace = True)
data = df_1_scaler[col]

In [ ]:
li = data.columns.tolist()
for i in range(2,len(li)):
    li[i] = 'T*'+li[i]
data.columns = li
df_2 = df.loc[0:,:]
df_2 = df_2.drop('Material Group',axis=1,inplace=False)
df_2 = df_2.drop('Method',axis=1,inplace=False)
df_2 = df_2.drop('Lattice form',axis=1,inplace=False)
df_2 = df_2.drop('Type',axis = 1,inplace = False)
df_2 = df_2.drop('S（uV/K）',axis = 1,inplace = False)
formula2 = df_2.pop('Formula')

In [ ]:
#数据归一化
from sklearn.preprocessing import MinMaxScaler

features_columns = [col for col in df_2.columns if col not in ['ZT']]
min_max_scaler = MinMaxScaler()
min_max_scaler = min_max_scaler.fit(df_2[features_columns])
df_2_scaler = min_max_scaler.transform(df_2[features_columns])

df_2_scaler = pd.DataFrame(df_2_scaler)
df_2_scaler.columns = features_columns


df_2_scaler['ZT'] = df_2['ZT']

In [ ]:
k = ['PymatgenData','DemlData','MagpieData']
for i in range(len(df_2_scaler.columns)):
    s = df_2_scaler.columns[i]
    s = s.split(' ')
    if s[0] in k:
        s = s[1:]
        s = ' '.join(s)
        df_2_scaler.rename(columns={df_2_scaler.columns[i]:s},inplace = True)
df_2_scaler2 = df_2_scaler[col]
df_2_scaler2 = df_2_scaler2.iloc[:,2:]
data =  pd.concat([data,df_2_scaler2],axis = 1)

In [ ]:
data1 = pd.concat([formula,data],axis = 1)
data2 = pd.concat([data,df1],axis = 1)
data3 = pd.concat([data2,type_],axis = 1)
testdata = data3.loc[5035:,:]
testdata = testdata.reset_index(drop=True)
df_data = data3.loc[:5034,:]

In [ ]:
import random
l = list(df_data.index) 
random.seed(2)
random.random()
random.shuffle(l) 
df_data.index = l

In [ ]:
n = df_data.shape[0]
m = int(n * 0.15)
test_data = df_data.loc[range(m), :] 
train_data = df_data.loc[range(m, n), :]
train_data = train_data.reset_index(drop=True)
all_data = pd.concat([test_data,train_data],axis = 0)
train_data = all_data.reset_index(drop=True)
long = df_data.iloc[:,358:]

In [ ]:
train_data = train_data.reset_index(drop=True)
pd.set_option('display.max_columns',10)
pd.set_option('display.max_rows',10)
data_train1 = train_data#.drop(['mean NsValence' , 'mean NfUnfilled', 'avg_dev NfUnfilled', 'mean NsUnfilled'], 
#                                   axis=1)
train_corr = data_train1.corr()
train_corr

In [ ]:
k = 10
cols = train_corr.nlargest(k, 'ZT')['ZT'].index

cm = np.corrcoef(train_data[cols].values.T)
hm = plt.subplots(figsize=(10,10))
hm = sns.heatmap(train_data[cols].corr(), annot=True, square=True)
plt.show()

In [ ]:
pd.set_option('display.max_columns',10)
pd.set_option('display.max_rows',10)
data_train2 = train_data#.drop(['mean NsValence' , 'mean NfUnfilled', 'avg_dev NfUnfilled', 'mean NsUnfilled'], 
#                               axis=1)
train_corr = data_train2.corr()
train_corr

In [ ]:
for i in range(train_corr.shape[0]):
    for j in range(train_corr.shape[1]):
        if 0.90 <abs(train_corr.iloc[i:i+1,j:j+1].values) < 1:
            print(i,j)

In [ ]:
m = []
for i in range(train_corr.shape[0]):
    for j in range(train_corr.shape[1]):
        if 0.90 <abs(train_corr.iloc[i:i+1,j:j+1].values) < 1:
            if abs(train_corr.iloc[i:i+1,0:1].values) < abs(train_corr.iloc[j:j+1,0:1].values):
                if train_corr.index[i] not in m:
                    m.append(train_corr.index[i])
            elif abs(train_corr.iloc[i:i+1,0:1].values) > abs(train_corr.iloc[j:j+1,0:1].values):
                if train_corr.index[j] not in m:
                    m.append(train_corr.index[j])
del_feature = m
del_feature

In [ ]:
for i in del_feature:
    train_data = train_data.drop(i,axis = 1,inplace = False)
train_data

In [ ]:
plt.figure(figsize=(20,16))
column =train_data.columns.tolist()
mcorr = train_data[column].corr(method='pearson')
mask = np.zeros_like(mcorr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True    
#cmap = sns.diverging_palette(220, 20, sep=20, as_cmap=True)  
g = sns.heatmap(mcorr, mask=mask,  square=True,
                annot=True, fmt='0.2f')
plt.show()

In [ ]:
mcorr = mcorr.abs()
numerical_corr=mcorr[mcorr['ZT'] > 0.1]['ZT']
print(numerical_corr.sort_values(ascending=False))
long_train = train_data.iloc[:,223:]
long_train

In [ ]:
long_test = test_data.iloc[:,222:]
long_test

In [ ]:
train_data_scaler = train_data
test_data_scaler = test_data[[col for col in train_data_scaler.columns if col not in ['ZT']]]
train_data_scaler.shape,test_data_scaler.shape

In [ ]:
labels = train_data_scaler.pop('ZT')

num = len(train_data_scaler.columns)
LGBM_ = LGBMRegressor(n_estimators=1000)

In [ ]:
score_ = []
for i in range(1,num,10):
    X_wrapper = RFE(LGBM_,n_features_to_select=i, step=10).fit_transform(train_data_scaler,labels)
    once = cross_val_score(LGBM_,X_wrapper,labels,cv=5,scoring='r2').mean()
    score_.append(once)

In [ ]:
x=1
score_1=[]
for i in score_:
    a=1-(1-i)*1006/(1006-x)
    x+=10
    print(a,x-10)
    score_1.append(a)
plt.figure(figsize=[16,6],dpi=100)
plt.plot(range(11,231,10), score_1[1:23], '-')
plt.xticks(range(11,231,10))
plt.grid()
plt.show()

In [ ]:
%%time
score_2 = []
for i in range(51,71,1):
    X_wrapper = RFE(LGBM_,n_features_to_select=i, step=1).fit_transform(train_data_scaler,labels)
    once = cross_val_score(LGBM_,X_wrapper,labels,cv=5,scoring='r2').mean()
    score_2.append(once)

In [ ]:
x=51
score_3=[]
for i in score_2:
    a=1-(1-i)*1006/(1006-x)
    x+=1
    print(a,x-1)
    score_3.append(a)
plt.figure(figsize=[16,6],dpi=100)
plt.plot(range(51,71,1), score_3[0:20], '-')
plt.xticks(range(51,71,1))
plt.grid()
plt.show()

In [ ]:
selector = RFE(LGBM_,n_features_to_select=57,step=1).fit(train_data_scaler,labels)
X_wrapper = selector.transform(train_data_scaler)
cross_val_score(LGBM_,X_wrapper,labels,cv=5,scoring='neg_mean_squared_error').mean()
#取出被选定的特征
select = []
for i in range(len(selector.support_)):
    if selector.support_[i]:
        select.append(train_data_scaler.columns.values[i])

select

In [ ]:
select.append('ZT')

In [ ]:
train_data_corr = train_data[select]
label = train_data_corr['ZT']
label

# Modeling

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import SGDRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost.sklearn import XGBRegressor
from lightgbm import LGBMRegressor


from sklearn.model_selection import GridSearchCV
import joblib
from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score
from sklearn.model_selection import KFold,train_test_split

from sklearn.model_selection import ShuffleSplit, learning_curve

import os
import time
import warnings

warnings.filterwarnings('ignore')

In [ ]:
train = train_data.iloc[:,1:].values
target = train_data['ZT'].values
X_train, X_test, y_train, y_test = train_test_split(train, target, test_size=0.2, random_state=0)
X_train.shape,X_test.shape

In [ ]:
X_train = pd.DataFrame(X_train)
X_test = pd.DataFrame(X_test)
X_train,X_test
k=['56']

In [ ]:
clf = LGBMRegressor(learning_rate=0.1,
                    max_depth=-1,
                    n_estimators=3000,
                    boosting_type='gbdt',
                    random_state=2021,
                    objective='regression')

clf.fit(X=X_train, y=y_train, eval_metric='MSE', verbose=50,categorical_feature=k)
score = mean_squared_error(y_test, clf.predict(X_test))
print('LGBMRegressor:', score)

In [ ]:
Folds = 10
kf = KFold(n_splits=Folds,shuffle=True, random_state=0)

train_pred = np.zeros((train.shape[0], ))
MSE_DICT = {'Train_mse': [], 'Test_mse': []}

In [ ]:
for i, (train_index, test_index) in enumerate(kf.split(train)):
    
    lgb_reg = LGBMRegressor(max_depth=-1,
                            n_estimators=1300,
                            num_leaves=15,
                            subsample=0.6,
                            bagging_fraction=0.5,
                            colsample_bytree=0.6,
                            feature_fraction=1.0,
                            random_satate=2021,
                            min_child_weight=0.001,
                            min_child_samples=20,
                            objective='regression')
    
    
    X_train_KFold, X_test_KFold = pd.DataFrame(train[train_index]), pd.DataFrame(train[test_index])
    y_train_KFold, y_test_KFold = target[train_index], target[test_index]
    
    
    lgb_reg.fit(X=X_train_KFold,
                y=y_train_KFold,
                eval_set=[(X_train_KFold,y_train_KFold),
                          (X_test_KFold,y_test_KFold)],
                eval_names=['Train','Test'],
                early_stopping_rounds=150,
                eval_metric='RMSE',
                verbose=50,categorical_feature = k
               )
    
    
    y_train_KFold_predict = abs(lgb_reg.predict(X_train_KFold, num_iteration=lgb_reg.best_iteration_))
    y_test_KFold_predict = abs(lgb_reg.predict(X_test_KFold, num_iteration=lgb_reg.best_iteration_))
    
    
    print('第{}折 训练和预测：'.format(i+1))
    train_mse = np.sqrt(mean_squared_error(y_train_KFold_predict,y_train_KFold))
    print('--------\n', '训练集RMSE\n', train_mse, '\n--------')
    test_mse = np.sqrt(mean_squared_error(y_test_KFold_predict,y_test_KFold))
    print('--------\n', '测试集RMSE\n', test_mse, '\n--------\n')
    
    MSE_DICT['Train_mse'].append(train_mse)
    MSE_DICT['Test_mse'].append(test_mse)
    

    
    print("记录测试集预测结果...")
    train_pred[test_index] = abs(lgb_reg.predict(X_test_KFold, num_iteration=lgb_reg.best_iteration_))
    

print('--------\n', '总训练集RMSE\n', MSE_DICT['Train_mse'], '\n', '平均RMSE：{}'.format(np.mean(MSE_DICT['Train_mse'])), '\n')
print('--------\n', '总测试集RMSE\n', MSE_DICT['Test_mse'], '\n', '平均RMSE：{}'.format(np.mean(MSE_DICT['Test_mse'])), '\n')

In [ ]:
train_data.iloc[:,1:].columns
feature_names=train_data.iloc[:,1:].columns
lgb_reg.feature_importances_
data_importance=pd.DataFrame({
        'Features': feature_names,
        'Importance': lgb_reg.feature_importances_,
    }).sort_values(by='Importance')
data_importance=data_importance.nlargest(20,'Importance')
data_importance=data_importance.reset_index(drop=True)
data_importance

In [ ]:
Folds = 10
kf = KFold(n_splits=Folds,shuffle=True, random_state=0)

train_pred1 = np.zeros((train.shape[0], ))
MSE_DICT = {'Train_mse': [], 'Test_mse': []} 

In [ ]:

for i, (train_index, test_index) in enumerate(kf.split(train)):
   
    xgb_reg = xgb_reg = XGBRegressor(learning_rate=0.1,
                        max_depth=9,
                        n_estimators=3000,
                        booster='gbtree',
                        random_state=2021,
                        objective='reg:squarederror',
                        min_child_weight=10,      
                            subsample=0.5,              
                            colsample_bytree=1)
    
 
    X_train_KFold, X_test_KFold = train[train_index], train[test_index]
    y_train_KFold, y_test_KFold = target[train_index], target[test_index]
    
 
    xgb_reg.fit(X=X_train_KFold,
                y=y_train_KFold,
                )
    

    y_train_KFold_predict = abs(xgb_reg.predict(X_train_KFold))
    y_test_KFold_predict = abs(xgb_reg.predict(X_test_KFold))
    

    print('第{}折 训练和预测：'.format(i+1))
    train_mse =np.sqrt(mean_squared_error(y_train_KFold_predict,y_train_KFold))
    print('--------\n', '训练集RMSE\n', train_mse, '\n--------')
    test_mse = np.sqrt(mean_squared_error(y_test_KFold_predict,y_test_KFold))
    print('--------\n', '测试集RMSE\n', test_mse, '\n--------\n')
    
    MSE_DICT['Train_mse'].append(train_mse)
    MSE_DICT['Test_mse'].append(test_mse)
    
    print("记录测试集预测结果...")
    train_pred[test_index] = abs(xgb_reg.predict(X_test_KFold))
    

print('--------\n', '总训练集RMSE\n', MSE_DICT['Train_mse'], '\n', '平均RMSE：{}'.format(np.mean(MSE_DICT['Train_mse'])), '\n')
print('--------\n', '总测试集RMSE\n', MSE_DICT['Test_mse'], '\n', '平均RMSE：{}'.format(np.mean(MSE_DICT['Test_mse'])), '\n')

In [ ]:
Folds = 10
kf = KFold(n_splits=Folds,shuffle=True, random_state=0)

train_pred1 = np.zeros((train.shape[0], ))
MSE_DICT = {'Train_mse': [], 'Test_mse': []} 

In [ ]:
for i, (train_index, test_index) in enumerate(kf.split(train)):
    
    knn_reg = KNeighborsRegressor(n_neighbors=2)
    

    X_train_KFold, X_test_KFold = train[train_index], train[test_index]
    y_train_KFold, y_test_KFold = target[train_index], target[test_index]
    

    knn_reg.fit(X=X_train_KFold,
                y=y_train_KFold,
                )
    

    y_train_KFold_predict = abs(knn_reg.predict(X_train_KFold))
    y_test_KFold_predict = abs(knn_reg.predict(X_test_KFold))
    

    print('第{}折 训练和预测：'.format(i+1))
    train_mse =np.sqrt(mean_squared_error(y_train_KFold_predict,y_train_KFold))
    print('--------\n', '训练集MAE\n', train_mse, '\n--------')
    test_mse = np.sqrt(mean_squared_error(y_test_KFold_predict,y_test_KFold))
    print('--------\n', '测试集MAE\n', test_mse, '\n--------\n')
    
    MSE_DICT['Train_mse'].append(train_mse)
    MSE_DICT['Test_mse'].append(test_mse)
    


    print("记录测试集预测结果...")
    train_pred[test_index] = abs(knn_reg.predict(X_test_KFold))
    

print('--------\n', '总训练集RMSE\n', MSE_DICT['Train_mse'], '\n', '平均RMSE：{}'.format(np.mean(MSE_DICT['Train_mse'])), '\n')
print('--------\n', '总测试集RMSE\n', MSE_DICT['Test_mse'], '\n', '平均RMSE：{}'.format(np.mean(MSE_DICT['Test_mse'])), '\n')

In [ ]:
Folds = 10
kf = KFold(n_splits=Folds,shuffle=True, random_state=0)

train_pred = np.zeros((train.shape[0], ))
MSE_DICT = {'Train_mse': [], 'Test_mse': []}

In [ ]:
for i, (train_index, test_index) in enumerate(kf.split(train)):

    rf_reg = RandomForestRegressor(max_depth=10,
                                         n_estimators=1200, 
                                         random_state=2020,
                                         n_jobs=4,
                                         max_features='auto',
                                         verbose=1,
                                         min_samples_split=2,
                                        min_samples_leaf=1,
                                        min_weight_fraction_leaf=0.0,)
    

    X_train_KFold, X_test_KFold = train[train_index], train[test_index]
    y_train_KFold, y_test_KFold = target[train_index], target[test_index]
    

    rf_reg.fit(X=X_train_KFold,
                y=y_train_KFold,
                )
    

    y_train_KFold_predict = abs(rf_reg.predict(X_train_KFold))
    y_test_KFold_predict = abs(rf_reg.predict(X_test_KFold))
    

    print('第{}折 训练和预测：'.format(i+1))
    train_mse = np.sqrt(mean_squared_error(y_train_KFold_predict,y_train_KFold))
    print('--------\n', '训练集RMSE\n', train_mse, '\n--------')
    test_mse = np.sqrt(mean_squared_error(y_test_KFold_predict,y_test_KFold))
    print('--------\n', '测试集RMSE\n', test_mse, '\n--------\n')
    
    MSE_DICT['Train_mse'].append(train_mse)
    MSE_DICT['Test_mse'].append(test_mse)
    

    print("记录测试集预测结果...")
    train_pred[test_index] = abs(rf_reg.predict(X_test_KFold))
    
#总验证结果
print('--------\n', '总训练集RMSE\n', MSE_DICT['Train_mse'], '\n', '平均RMSE：{}'.format(np.mean(MSE_DICT['Train_mse'])), '\n')
print('--------\n', '总测试集RMSE\n', MSE_DICT['Test_mse'], '\n', '平均RMSE：{}'.format(np.mean(MSE_DICT['Test_mse'])), '\n')

In [ ]:
Folds = 10
kf = KFold(n_splits=Folds,shuffle=True, random_state=0)

train_pred = np.zeros((train.shape[0], ))
MSE_DICT = {'Train_mse': [], 'Test_mse': []}

In [ ]:
for i, (train_index, test_index) in enumerate(kf.split(train)):
    
    dt_reg = DecisionTreeRegressor(min_samples_split=2,
                                        min_samples_leaf=1,
                                        min_weight_fraction_leaf=0.0,
                                        max_features='auto',
                                        random_state=2021,
                                        max_depth=11)
    
   
    X_train_KFold, X_test_KFold = train[train_index], train[test_index]
    y_train_KFold, y_test_KFold = target[train_index], target[test_index]
    

    dt_reg.fit(X=X_train_KFold,
                y=y_train_KFold,
                )
    

    y_train_KFold_predict = abs(dt_reg.predict(X_train_KFold))
    y_test_KFold_predict = abs(dt_reg.predict(X_test_KFold))
    

    print('第{}折 训练和预测：'.format(i+1))
    train_mse = np.sqrt(mean_squared_error(y_train_KFold_predict,y_train_KFold))
    print('--------\n', '训练集MAE\n', train_mse, '\n--------')
    test_mse = np.sqrt(mean_squared_error(y_test_KFold_predict,y_test_KFold))
    print('--------\n', '测试集MAE\n', test_mse, '\n--------\n')
    
    MSE_DICT['Train_mse'].append(train_mse)
    MSE_DICT['Test_mse'].append(test_mse)
    


    print("记录测试集预测结果...")
    train_pred[test_index] = abs(dt_reg.predict(X_test_KFold))
    

print('--------\n', '总训练集RMSE\n', MSE_DICT['Train_mse'], '\n', '平均RMSE：{}'.format(np.mean(MSE_DICT['Train_mse'])), '\n')
print('--------\n', '总测试集RMSE\n', MSE_DICT['Test_mse'], '\n', '平均RMSE：{}'.format(np.mean(MSE_DICT['Test_mse'])), '\n')

In [ ]:
testdata1 = testdata[train_data.columns]
testdata2 = testdata1.iloc[:,1:]
testdata2

In [ ]:
y_test_predict1 = abs(lg.predict(testdata2.values))